# 看護師離職率データセット作成（ETL完全版）

このノートブックは、手作業によるデータ作成と同等の品質（正確な数値、適切な単位）を自動化するためのものです。
特に**「沖縄県のデータ異常」「年収の単位変換」「家賃の民営借家指定」**などの重要ロジックが実装されています。

## 前提
- 元データ（CSV/Excel）が `../data/raw/` に配置されていること。
- 出力先は `../data/processed/` です。

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

# ==========
# Project paths (ディレクトリ構造に合わせて修正)
# ==========

# ノートブックの場所（実行コンテキスト）に依存しない、プロジェクトルートの絶対パス解決
PROJECT_ROOT = Path("..").resolve()

# データレイク（ストレージ層）のルートディレクトリ定義
DATA_DIR = PROJECT_ROOT / "data"

# Landing Zone（Bronze層）：ソース源泉の生データ（Immutable/変更不可）格納パス
RAW_DIR = DATA_DIR / "raw"

# Staging Area（Silver層）：クレンジング・正規化・構造化済みの中間データ格納パス
PROCESSED_DIR = DATA_DIR / "clean"

# Production Area（Gold層）：分析用マート（ABT）や最終成果物（Artifact）格納パス
OUT_DIR = DATA_DIR / "out"

# パイプラインの冪等性（Idempotency）を担保したStagingディレクトリのプロビジョニング
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# 出力先ディレクトリの存在保証（再実行時のIOエラー回避・フォールトトレランス向上）
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ==========
# Debug Check (パスが合っているか確認)
# ==========
print(f"プロジェクトルート: {PROJECT_ROOT}")   # 環境変数・パス解決結果の監査ログ出力
print(f"読み込み元 (Raw): {RAW_DIR}")       # 入力ソースパス（Source Path）の整合性確認
print(f"保存先 (Processed): {PROCESSED_DIR}") # 中間生成物パス（Intermediate Path）の確認
print(f"保存先 (Out): {OUT_DIR}")           # 出力先パス（Destination Path）の確認

プロジェクトルート: /Users/hideomi.h/nurse-turnover-analysis
読み込み元 (Raw): /Users/hideomi.h/nurse-turnover-analysis/data/raw
保存先 (Processed): /Users/hideomi.h/nurse-turnover-analysis/data/clean
保存先 (Out): /Users/hideomi.h/nurse-turnover-analysis/data/out


In [2]:
# ==========================
# 1-1 Rawデータ確認（turnover）
# ==========================

# 1. 許可リスト（Whitelist）の作成
# 取り込み（Ingestion）を許可するファイル形式を「OKリスト」として定義
ALLOWED_EXT = {".csv", ".xlsx", ".xls", ".pdf"}

# 2. ファイルの選別（Filtering）と整列（Sorting）
# 「いつ実行しても同じ結果（Idempotency）」にするため、名前順に並べ替えて確定
raw_files = sorted(
    # rawフォルダ（Landing Zone）にある全ファイルを一つずつチェック（Scan）
    p for p in RAW_DIR.glob("*")
    # 拡張子の「大文字・小文字」のズレを直し（Normalize）、OKリストにあるものだけを通す
    if p.suffix.lower() in ALLOWED_EXT
)

# 3. 最終確認（Audit）
# 処理するファイルの「数」と「名前」に、おかしな点がないか目視で検品
len(raw_files), [p.name for p in raw_files]

(13,
 ['厚労省_一般職業紹介状況_有効求人倍率_2025.xlsx',
  '厚労省_医療施設調査_病床規模_2024.csv',
  '厚労省_医療施設調査_病院数_2024.csv',
  '厚労省_衛生行政報告例_看護師数_2023.csv',
  '厚労省_賃金構造基本統計調査_看護師年収_2024.xlsx',
  '日本看護協会_夜勤72h超過率_都道府県別_2024.csv',
  '日本看護協会_病院看護実態調査_離職率等_2025.pdf',
  '日本看護協会_給与_都道府県別_2024.csv',
  '日本看護協会_離職率_都道府県別_2023.csv',
  '総務省_住宅土地統計調査_家賃_2024.xlsx',
  '総務省_住宅土地統計調査_持ち家率_2024.xlsx',
  '総務省_社会生活基本調査_通勤時間_2022.xlsx',
  '総務省_社会生活統計指標_人口密度_2023.csv'])

In [3]:
# ==========================
# 1-2a 対象ファイル指定（turnover）
# ==========================

# Ingestion（取り込み）対象とする特定のSource Asset（データ源泉）を絶対パスで定義
raw_turnover_path = RAW_DIR / "日本看護協会_離職率_都道府県別_2023.csv"

# 処理実行ログ（Execution Log）への出力による、パイプラインのTraceability（追跡可能性）の確保
print("target:", raw_turnover_path)

target: /Users/hideomi.h/nurse-turnover-analysis/data/raw/日本看護協会_離職率_都道府県別_2023.csv


In [4]:
# ==========================
# 1-2b 存在・サイズチェック（pre-check）
# ==========================

# Source Assetの存在確認（Sanity Check）：ファイル未到達によるパイプラインの異常停止を防止
assert raw_turnover_path.exists(), "❌ ファイルが存在しません"

# Storage層からファイルのMetadata（メタデータ）を抽出し、ファイルサイズを取得
size_bytes = raw_turnover_path.stat().st_size

# 下流工程（Downstream）での可読性向上のため、バイト単位をKBに単位変換（Normalization）
size_kb = size_bytes / 1024

# 実行ログ（Observability）にファイルの状態を記録し、トレーサビリティを確保
print(f"✅ exists. size={size_kb:.1f} KB")

# データ破損（Data Corruption）の検知：0バイトファイルを排除するバリデーション
assert size_bytes > 0, "❌ ファイルサイズが0です（空ファイル）"

✅ exists. size=1.2 KB


In [5]:
# ==========================
# 1-2c 先頭バイト確認（encoding推定）
# ==========================

# Encoding不一致によるパースエラーを防ぐため、ファイルをバイナリモード（rb）でサンプリング
with open(raw_turnover_path, "rb") as f:
    # ファイル先頭の16バイト（Magic Bytes/ヘッダ情報）をメモリにロード
    head = f.read(16)

# 推論（Inference）したバイナリデータをログ出力し、文字コードのプロファイリングを実施
print("head bytes:", head)

head bytes: b'\xef\xbb\xbfprefecture,tu'


In [6]:
# ==========================
# 1-2d 確定読み込み（UTF-8 with BOM）
# ==========================

# BOMを自動処理（Strip）するエンコーディングを指定し、RawアセットをDataFrameとしてロード
df_raw_turnover = pd.read_csv(
    raw_turnover_path,
    encoding="utf-8-sig"
)

# 実行ログ（Audit Log）にデプロイ済みのデコード方式を記録
print("loaded with: utf-8-sig")

# インジェスト後のデータボリューム（行列数）をプロファイリング
print("shape:", df_raw_turnover.shape)

# データ構造（Schema）とレコードの整合性を視覚的にインスペクション（目視確認）
display(df_raw_turnover.head())

# 下流工程（Downstream）のコーディング用に、カラムリスト（Metadata）を抽出
display(df_raw_turnover.columns.tolist())

loaded with: utf-8-sig
shape: (47, 4)


,prefecture,turnover_total,turnover_new_grad,turnover_experienced
0,北海道,11.5,5.9,16.6
1,青森県,8.6,10.7,16.7
2,岩手県,6.8,7.8,19.1
3,宮城県,9.1,7.1,12.4
4,秋田県,7.4,5.0,7.3


['prefecture', 'turnover_total', 'turnover_new_grad', 'turnover_experienced']

In [7]:
# ==========================
# 2-1a turnover 前処理（interim化）
# ==========================

# データの非破壊性を担保するため、Rawデータのディープコピー（Deep Copy）を作成
df_interim_turnover = df_raw_turnover.copy()

# 結合や集計時のKey不一致を防ぐため、特定の次元（Dimension）に対してトリミング処理（Strip）を実行
df_interim_turnover["prefecture"] = df_interim_turnover["prefecture"].astype("string").str.strip()

# 処理対象とする必須項目のみを抽出し、メモリ効率とスキーマ（Schema）の透明性を確保
required_cols = ["prefecture", "turnover_total", "turnover_new_grad", "turnover_experienced"]
df_interim_turnover = df_interim_turnover[required_cols].copy()

# データ型（Data Type）の強制適用を行い、不正な文字列（Dirty Data）を強制的に欠損値（NaN）へ変換
for c in ["turnover_total", "turnover_new_grad", "turnover_experienced"]:
    df_interim_turnover[c] = pd.to_numeric(df_interim_turnover[c], errors="coerce")

# 下流工程での年次比較やデータ統合（Union）に備え、全レコードに2023年度の識別子を付与
df_interim_turnover["fiscal_year"] = 2023

In [8]:
# ==========================
# 3-1 検証（interim_turnover）
# ==========================
# データボリューム（Data Volume）の監査ログ出力
print("rows:", len(df_interim_turnover))

# 必須項目（Required Columns）の完全性（Completeness）を確認するNullityチェック
nulls = df_interim_turnover[["prefecture","fiscal_year","turnover_total"]].isna().sum()
print("nulls:\n", nulls)

# 複合主キー（Composite Primary Key）の一意性制約（Uniqueness Constraint）の検証
dup = df_interim_turnover.duplicated(["prefecture","fiscal_year"]).sum()
print("dup_keys:", dup)

# 指標（Rate）が定義域（0-100%）に収まっているかの境界値チェック（Boundary Check）
# バリデーション対象とする指標（Rate）のリストを定義
rate_cols = ["turnover_total", "turnover_new_grad", "turnover_experienced"]
# 全対象カラムに対し、定義域（0-100%）逸脱を判定するBoolean Maskを生成して横結合（Horizontal Concat）
bad = pd.concat(
    [(df_interim_turnover[c] < 0) | (df_interim_turnover[c] > 100) for c in rate_cols],
    axis=1
# 行方向（axis=1）に走査し、いずれかのカラムで異常があればTrueとするSeriesを集約
).any(axis=1)
# 異常値を含むレコードの総数を集計し、オブザーバビリティ（可視性）を確保
print("out_of_range:", bad.sum())

# 品質基準（Quality Standards）を強制するハードガード（Assertion）
# レコード数が都道府県数（47）と完全一致することを確認（欠落や過剰の検知）
assert len(df_interim_turnover) == 47
# 一意性制約の不備（重複）が1件も存在しないことを断言
assert dup == 0
# 必須項目の完全性（Null不在）を最終確認
assert nulls.sum() == 0
# 定義域（0-100%）を逸脱した異常値が混入していないことを保証
assert bad.sum() == 0

# インスペクション（目視確認）用のデータプレビュー
display(df_interim_turnover.head(5))

rows: 47
nulls:
 prefecture        0
fiscal_year       0
turnover_total    0
dtype: int64
dup_keys: 0
out_of_range: 0


,prefecture,turnover_total,turnover_new_grad,turnover_experienced,fiscal_year
0,北海道,11.5,5.9,16.6,2023
1,青森県,8.6,10.7,16.7,2023
2,岩手県,6.8,7.8,19.1,2023
3,宮城県,9.1,7.1,12.4,2023
4,秋田県,7.4,5.0,7.3,2023


In [9]:
# ==========================
# 1-X Validation（turnover）
# ==========================

print("rows:", len(df_interim_turnover))

# KEY重複チェック
dup = df_interim_turnover.duplicated(["prefecture","fiscal_year"]).sum()
print("duplicate keys:", dup)

# NULLチェック（必須列）
for c in [
    "turnover_total",
    "turnover_new_grad",
    "turnover_experienced",
]:
    print(f"null {c}:", df_interim_turnover[c].isna().sum())

# 値域チェック（%）
for c in [
    "turnover_total",
    "turnover_new_grad",
    "turnover_experienced",
]:
    out = ((df_interim_turnover[c] < 0) | (df_interim_turnover[c] > 100)).sum()
    print(f"{c} out of range:", out)


rows: 47
duplicate keys: 0
null turnover_total: 0
null turnover_new_grad: 0
null turnover_experienced: 0
turnover_total out of range: 0
turnover_new_grad out of range: 0
turnover_experienced out of range: 0


In [10]:
# ==========================
# 4-1 保存（interim_turnover）
# ==========================
# 物理保存先のパス（URI）を定義し、永続化レイヤーのディレクトリを指定
interim_turnover_path = PROCESSED_DIR / "interim_turnover.parquet"

# メモリ上のDataFrameを、I/O効率の良い列指向フォーマットでストレージへフラッシュ
df_interim_turnover.to_parquet(interim_turnover_path, index=False)

# 保存完了のログを出力し、パイプラインの実行ステータスを可視化
print("saved:", interim_turnover_path)

saved: /Users/hideomi.h/nurse-turnover-analysis/data/clean/interim_turnover.parquet


In [11]:
# ==========================
# 1-2a 対象ファイル指定（night_shift）
# ==========================
raw_ns_path = RAW_DIR / "日本看護協会_夜勤72h超過率_都道府県別_2024.csv"
print("target:", raw_ns_path)

target: /Users/hideomi.h/nurse-turnover-analysis/data/raw/日本看護協会_夜勤72h超過率_都道府県別_2024.csv


In [12]:
# ==========================
# 1-2b 存在・サイズチェック
# ==========================
assert raw_ns_path.exists(), "❌ ファイルが存在しません"
size_bytes = raw_ns_path.stat().st_size
print(f"✅ exists. size={size_bytes/1024:.1f} KB")
assert size_bytes > 0, "❌ 空ファイルです"

✅ exists. size=1.2 KB


In [13]:
# ==========================
# 1-2c 先頭バイト確認
# ==========================
with open(raw_ns_path, "rb") as f:
    head = f.read(16)
print("head bytes:", head)

head bytes: b'\xef\xbb\xbfprefecture,ni'


In [14]:
# ==========================
# 1-2d 確定読み込み（UTF-8 with BOM）
# ==========================

df_raw_ns = pd.read_csv(
    raw_ns_path,
    encoding="utf-8-sig"
)

print("✅ loaded with: utf-8-sig")
print("shape:", df_raw_ns.shape)
display(df_raw_ns.head(5))
display(df_raw_ns.columns.tolist())


✅ loaded with: utf-8-sig
shape: (47, 4)


,prefecture,night_shift_72h_plus,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift
0,北海道,36.7,7.8,4.6
1,青森県,36.5,7.7,4.8
2,岩手県,11.8,7.5,4.1
3,宮城県,30.2,8.0,4.7
4,秋田県,25.1,7.7,4.3


['prefecture',
 'night_shift_72h_plus',
 'night_shifts_per_month_three_shift',
 'night_shifts_per_month_two_shift']

In [15]:
# ==========================
# 2-1a interim化（night_shift）
# ==========================
# Rawデータの破壊を防止し、副作用（Side Effect）を遮断するためのディープコピー
df_interim_night_shift = df_raw_ns.copy()

# 接合キーの不一致を防ぐためのサニタイズ（型固定と空白除去）
df_interim_night_shift["prefecture"] = df_interim_night_shift["prefecture"].astype("string").str.strip()

# 数値変換のバリデーション対象（Metrics）をリストで定義
num_cols = [
    "night_shift_72h_plus",
    "night_shifts_per_month_three_shift",
    "night_shifts_per_month_two_shift",
]

# 非数値混入によるクラッシュを防ぐための安全なキャスト（Coercion）
for c in num_cols:
    df_interim_night_shift[c] = pd.to_numeric(df_interim_night_shift[c], errors="coerce")

# 分析の主軸（Partition Key）となる年度情報を定数として付与
df_interim_night_shift["fiscal_year"] = 2023

# データの出所（Data Lineage）を明示するための参照年を保持
df_interim_night_shift["reference_year"] = 2024

# 必要なカラムのみを射影（Projection）し、物理的なメモリ配置を最適化
df_interim_night_shift = df_interim_night_shift[
    ["prefecture", "fiscal_year", "reference_year"] + num_cols
].copy()


In [16]:
# ==========================
# 3-1 検証（interim_night_shift）
# ==========================
print("rows:", len(df_interim_night_shift))
print("dup_keys:", df_interim_night_shift.duplicated(["prefecture","fiscal_year"]).sum())

nulls = df_interim_night_shift[["prefecture","fiscal_year"] + num_cols].isna().sum()
print("nulls:\n", nulls)

# 値域チェック
bad_rate = (df_interim_night_shift["night_shift_72h_plus"] < 0) | (df_interim_night_shift["night_shift_72h_plus"] > 100)
print("out_of_range_rate:", bad_rate.sum())

bad_counts = (
    (df_interim_night_shift["night_shifts_per_month_three_shift"] < 0) |
    (df_interim_night_shift["night_shifts_per_month_two_shift"] < 0)
)
print("out_of_range_counts:", bad_counts.sum())

# OK判定
assert len(df_interim_night_shift) == 47
assert df_interim_night_shift.duplicated(["prefecture","fiscal_year"]).sum() == 0
assert nulls.sum() == 0
assert bad_rate.sum() == 0
assert bad_counts.sum() == 0

display(df_interim_night_shift.head(5))


rows: 47
dup_keys: 0
nulls:
 prefecture                            0
fiscal_year                           0
night_shift_72h_plus                  0
night_shifts_per_month_three_shift    0
night_shifts_per_month_two_shift      0
dtype: int64
out_of_range_rate: 0
out_of_range_counts: 0


,prefecture,fiscal_year,reference_year,night_shift_72h_plus,night_shifts_per_month_three_shift,night_shifts_per_month_two_shift
0,北海道,2023,2024,36.7,7.8,4.6
1,青森県,2023,2024,36.5,7.7,4.8
2,岩手県,2023,2024,11.8,7.5,4.1
3,宮城県,2023,2024,30.2,8.0,4.7
4,秋田県,2023,2024,25.1,7.7,4.3


In [17]:
# ==========================
# 1-X Validation（night_shift）
# ==========================

print("rows:", len(df_interim_night_shift))

# KEY重複チェック
dup = df_interim_night_shift.duplicated(["prefecture","fiscal_year"]).sum()
print("duplicate keys:", dup)

# NULLチェック
for c in [
    "night_shift_72h_plus",
    "night_shifts_per_month_three_shift",
    "night_shifts_per_month_two_shift"
]:
    print(f"null {c}:", df_interim_night_shift[c].isna().sum())

# 値域チェック
print("72h_plus out of range:",
      ((df_interim_night_shift["night_shift_72h_plus"] < 0) |
       (df_interim_night_shift["night_shift_72h_plus"] > 100)).sum())


rows: 47
duplicate keys: 0
null night_shift_72h_plus: 0
null night_shifts_per_month_three_shift: 0
null night_shifts_per_month_two_shift: 0
72h_plus out of range: 0


In [18]:
# ==========================
# 4-1 保存（interim_night_shift）
# ==========================
out_path = PROCESSED_DIR / "interim_night_shift.parquet"
df_interim_night_shift.to_parquet(out_path, index=False)
print("saved:", out_path)


saved: /Users/hideomi.h/nurse-turnover-analysis/data/clean/interim_night_shift.parquet


In [19]:
# ==========================
# 4-2 Read-back check
# ==========================
df_check = pd.read_parquet(out_path)

print("read-back shape:", df_check.shape)
print(df_check.columns.tolist())

read-back shape: (47, 6)
['prefecture', 'fiscal_year', 'reference_year', 'night_shift_72h_plus', 'night_shifts_per_month_three_shift', 'night_shifts_per_month_two_shift']


In [20]:
# ==========================
# 1-2a Target file（salary）
# ==========================
raw_salary_path = RAW_DIR / "日本看護協会_給与_都道府県別_2024.csv"
assert raw_salary_path.exists(), "❌ ファイルが存在しません"
print("target:", raw_salary_path)


target: /Users/hideomi.h/nurse-turnover-analysis/data/raw/日本看護協会_給与_都道府県別_2024.csv


In [21]:
# ==========================
# 1-2b File size check
# ==========================
size_bytes = raw_salary_path.stat().st_size
print(f"size: {size_bytes/1024:.1f} KB")
assert size_bytes > 0, "❌ 空ファイルです"


size: 1.8 KB


In [22]:
# ==========================
# 1-2c Head bytes check (encoding hint)
# ==========================
with open(raw_salary_path, "rb") as f:
    head = f.read(16)
print("head bytes:", head)

head bytes: b'\xef\xbb\xbfprefecture,st'


In [23]:
# ==========================
# 1-2d Read CSV（assume utf-8-sig）
# ==========================
df_raw_salary = pd.read_csv(raw_salary_path, encoding="utf-8-sig")
print("shape:", df_raw_salary.shape)
display(df_raw_salary.head(5))
display(df_raw_salary.columns.tolist())

shape: (47, 4)


,prefecture,starting_salary_nurse_diploma_monthly_yen,starting_salary_nurse_bachelor_monthly_yen,salary_nurse_10yr_non_manager_monthly_yen
0,北海道,270231.0,276652.0,326530.0
1,青森県,270494.0,275476.0,314135.0
2,岩手県,268264.0,276926.0,322466.0
3,宮城県,272385.0,280377.0,332670.0
4,秋田県,263109.0,272553.0,321717.0


['prefecture',
 'starting_salary_nurse_diploma_monthly_yen',
 'starting_salary_nurse_bachelor_monthly_yen',
 'salary_nurse_10yr_non_manager_monthly_yen']

In [24]:
# ==========================
# 2-1 Transform → interim化（salary）
# ==========================
df_interim_salary = df_raw_salary.copy()

# prefecture（標準化：最低限）
df_interim_salary["prefecture"] = (
    df_interim_salary["prefecture"]
    .astype("string")
    .str.strip()
)

# fiscal_year（結合キー）
df_interim_salary["fiscal_year"] = 2023

# reference_year（観測年）
df_interim_salary["reference_year"] = 2024

# 数値列
num_cols = [
    "starting_salary_nurse_diploma_monthly_yen",
    "starting_salary_nurse_bachelor_monthly_yen",
    "salary_nurse_10yr_non_manager_monthly_yen",
]

for c in num_cols:
    df_interim_salary[c] = pd.to_numeric(df_interim_salary[c], errors="coerce")

# 必要列だけに射影
keep_cols = [
    "prefecture",
    "fiscal_year",
    "reference_year",
] + num_cols

df_interim_salary = df_interim_salary[keep_cols].copy()

df_interim_salary.head()


,prefecture,fiscal_year,reference_year,starting_salary_nurse_diploma_monthly_yen,starting_salary_nurse_bachelor_monthly_yen,salary_nurse_10yr_non_manager_monthly_yen
0,北海道,2023,2024,270231.0,276652.0,326530.0
1,青森県,2023,2024,270494.0,275476.0,314135.0
2,岩手県,2023,2024,268264.0,276926.0,322466.0
3,宮城県,2023,2024,272385.0,280377.0,332670.0
4,秋田県,2023,2024,263109.0,272553.0,321717.0


In [25]:
# ==========================
# 3-1 Validation（salary）
# ==========================
print("rows:", len(df_interim_salary))

# 主キー重複
dup = df_interim_salary.duplicated(["prefecture", "fiscal_year"]).sum()
print("dup_keys:", dup)

# NULLチェック
check_cols = ["prefecture", "fiscal_year"] + num_cols
nulls = df_interim_salary[check_cols].isna().sum()
print("nulls:\n", nulls)

# 非負チェック（給与）
for c in num_cols:
    neg = (df_interim_salary[c] < 0).sum()
    print(f"{c} negative:", neg)

# assert（止める）
assert dup == 0
assert df_interim_salary[check_cols].isna().sum().sum() == 0
assert len(df_interim_salary) == 47

rows: 47
dup_keys: 0
nulls:
 prefecture                                    0
fiscal_year                                   0
starting_salary_nurse_diploma_monthly_yen     0
starting_salary_nurse_bachelor_monthly_yen    0
salary_nurse_10yr_non_manager_monthly_yen     0
dtype: int64
starting_salary_nurse_diploma_monthly_yen negative: 0
starting_salary_nurse_bachelor_monthly_yen negative: 0
salary_nurse_10yr_non_manager_monthly_yen negative: 0


In [26]:
# ==========================
# 4-1 Save（interim_salary）
# ==========================
out_path = PROCESSED_DIR / "interim_salary.parquet"

df_interim_salary.to_parquet(out_path, index=False)

print("saved:", out_path)


saved: /Users/hideomi.h/nurse-turnover-analysis/data/clean/interim_salary.parquet


In [27]:
# ==========================
# 4-2 Read-back check
# ==========================
df_check = pd.read_parquet(out_path)

print("read-back shape:", df_check.shape)
print(df_check.columns.tolist())

read-back shape: (47, 6)
['prefecture', 'fiscal_year', 'reference_year', 'starting_salary_nurse_diploma_monthly_yen', 'starting_salary_nurse_bachelor_monthly_yen', 'salary_nurse_10yr_non_manager_monthly_yen']


# ==========================================================
# プロジェクト：看護職員離職率データの Interim（中間層）構築
# ==========================================================

## 1. データ前処理（Data Cleaning / Sanitization）
生データ（Raw）を計算可能な形に整える「データの検品・加工」工程です。

| 処理内容 | 使用コード（例） | 実務上の役割・エンジニアの意図 |
| :--- | :--- | :--- |
| **非破壊性の確保** | `.copy()` | 元データを汚さず、副作用（Side Effect）を防ぐための「予備」作成。 |
| **型の厳格化** | `.astype("string")` | 曖昧な型を「文字列」に固定。欠損値（NA）の扱いを安全にする。 |
| **名寄せ・トリミング** | `.str.strip()` | 見えない空白を除去。結合（Join）失敗の「サイレント・エラー」を防ぐ。 |
| **スキーマ定義** | `required_cols` | 必要な列（必須項目）を厳選。メモリ節約とデータの契約を明示。 |
| **数値化（無害化）** | `errors="coerce"` | ゴミ混入を強制的に欠損値に変え、計算エラーを回避する「安全装置」。 |
| **メタデータ付与** | `df["year"] = 2023` | 全行に年度ラベルを付与（ブロードキャスト）。時系列分析への備え。 |

---

## 2. クオリティ・ゲート（Quality Gate / Validation）
データが「分析に値する品質」を満たしているか、自動で判定する検問所です。

### 🏥 看護の視点でのバリデーション
- **完全性（Completeness）**: `isna().sum()`
    - 意味：必要な項目（バイタル等）の記入漏れがないか確認。
- **一意性（Uniqueness）**: `duplicated()`
    - 意味：一人の患者（県×年度）に、二重の指示書（データ）が存在しないか確認。
- **正当性（Validity）**: 値域（0-100%）チェック
    - 意味：測定値が生物学的にあり得ない数値（境界値逸脱）を示していないか確認。

### 🛠️ テクニカル・ロジック解説
"""
| 最小単位                       | 文法名称             | 実務上の役割・意図                                   |
|:---------------------------|:-----------------|:--------------------------------------------|
| [ ... for c in rate_cols]   | リスト内包表記       | 3つの列に対して「同じチェック」を一度に行う効率化の魔法。       |
| (df[...] < 0) | (df[...] > 100) | 論理演算（OR）      | 0未満または100超過という、定義域外の異常値を炙り出す。 |
| pd.concat(..., axis=1)      | 関数（横結合）       | 3列分のチェック結果を、1つの「判定シート」にまとめる。     |
| .any(axis=1)                | メソッド（行単位判定） | 1行ずつ見て「どこか1箇所でも不合格（True）があったか？」を判定。 |
| bad                         | 変数（Bool Series） | 最終的に「この行は正常(False)」「異常(True)」のラベル列に。 |
| .sum()                      | メソッド（集計）       | Trueは「1」として集計。異常があった行の総数を可視化。 |
"""

---

## 3. エンジニアとしての「攻め」と「守り」
- **Fail-fast原則**: `assert` を使い、異常があれば後続処理を即座に停止させる。
- **自己文書化**: コード内に「なぜこの処理が必要か」をメモ（Docstring等）し、未来の自分やチームへの申し送りを徹底する。

In [28]:
最小単位	Python文法の名称	実務上の役割・意図
assert	キーワード（文）	「断言する」。右側の条件が False なら、即座にプログラムを強制停止（エラー）させます。
len(...) == 47	比較演算（等価）	「47個あるはず」という仕様と「現実」を突き合わせます。
== 0	比較演算（ゼロ比較）	異常（重複、Null、範囲外）が 「ゼロ（皆無）」 であることを絶対条件とします。
.sum()	メソッド（集計）	個別のエラーフラグを合算し、1箇所でも「1（異常）」があればアウトと判定します。
最小単位までの分解（デコンストラクション）初心者の方が100%理解できるよう、文法と役割をマッピングします。コード要素文法上の呼称実務上の役割interim_turnover_path変数名「データの住所」に名前をつけ、後で使いやすくします。=代入演算子右側の情報を左側の名前に「格納」します。PROCESSED_DIRPathオブジェクトフォルダの位置をプログラムが扱える形で保持します。/演算子（オーバーロード）フォルダ名とファイル名をガッチャンコして「フルパス」を作ります。"interim_turnover.parquet"文字列（String）ファイルの具体的な名前を指定します。df_interim_turnoverインスタンス（変数）加工が終わって、今すぐ保存したい「データそのもの」です。.ドット演算子「〜の」という意味。データが持っている機能（メソッド）を呼び出します。to_parquetメソッド「Parquet形式に変換して保存せよ」という具体的な命令です。(...)コール（呼び出し）命令を実行するためのカッコです。index=Falseキーワード引数「余計なインデックスは保存しなくていいよ」というオプション指定です。

SyntaxError: invalid character '（' (U+FF08) (340604935.py, line 2)